# Nested CV Training

In this notebook we'll explain, with examples, how to use this library that has been designed with the purpose of:
* Performing nested cross validation for model and hyperparameter selection and performance estimation.
* Optimizing not only model hyperparameters, but whole preprocessing pipelines. 
* Dealing with imbalanced multiclass classification problems. 
* Probability calibration of predictions. 

## Imports

In [1]:
import sys
sys.path.append('..')
import numpy as np, pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from skopt.space import Real, Integer, Categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from nestedcvtraining.api import find_best_model
from nestedcvtraining.switch_case import SwitchCase
from nestedcvtraining.under_sampling_classifier import UnderSamplingClassifier
from skopt import gbrt_minimize

## Dataset
Let's show how to use the library with a synthetic multiclass imbalanced dataset:

In [2]:
from sklearn.datasets import make_classification
X, y = make_classification(
    n_samples=800,
    n_features=20, 
    n_redundant=5, 
    n_informative=5, 
    n_classes=3, 
    n_clusters_per_class=3,
    flip_y=0.05,
    class_sep=0.7,
    weights=[0.8, 0.15, 0.05],
    random_state=42
)

In [3]:
from collections import Counter
Counter(y)

Counter({2: 53, 0: 621, 1: 126})

In [4]:
X.shape

(800, 20)

## Model definition

One important piece of the library is that it allows to define potentially very complex nested pipelines, with different hyperparameters to optimize. The addition of the `SwitchCase` makes it easier to define modular structures and combine them in a last step. 

If the individual steps are all `sklearn` `transformers` or `estimators`, then everything can be combined with full flexibility. However, if there are `resamplers`, some care needs to be taken. I will present two *architectures* that work well in case there are `resamplers`, and they should be flexible enough for most cases. 

### Architecture 1

In this idea, we ensemble the whole model as a chain of `SwitchCase` inside an `imblearn` `Pipeline` object. We must take care of two things: 
* In the component that does the `resampling`, in the cases that contain a `resampler` we cannot use any `Pipeline`. Only individual `resamplers` or `passthrough`. In other cases we have freedom. But it's cleaner if we allow only `resamplers` in this component.  
* In other components, we can use `compose` objects, including `Pipeline`, but only from `sklearn` and with no `resamplers` inside (this is because `imblearn` `Pipelines` cannot contain nested `Pipelines`). 

The search space is defined in a straighforward manner, taking into account only the *double underscore* convention for accessing params of nested objects. 

An example of this architecture would be as follows:

In [5]:
resampler = SwitchCase(
    cases=[
        (
            "resampler_1",
            SMOTE(k_neighbors=3)
        ),
        (
            "resampler_2",
            "passthrough"
        )
    ],
    switch="resampler_1"
)

preprocessor = SwitchCase(
    cases=[
        (
            "prep_1",
            SkPipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", PCA(n_components=5))
            ])
        ),
        (
            "prep_2",
            SkPipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", SelectKBest(mutual_info_classif, k=5)),
            ])
        ),
        (
            "prep_3",
            "passthrough"
        )
    ],
    switch="prep_1"
)

model = SwitchCase(
    cases=[
        (
            "model_1",
            LogisticRegression()
        ),
        (
            "model_2",
            RandomForestClassifier()
        )
    ],
    switch="model_1"
)

clf = ImbPipeline(
    [("resampler", resampler), ("preprocessor", preprocessor), ("model", model)]
)

search_space= [
    Categorical(["resampler_1", "resampler_2"], name="resampler__switch"),
    Categorical(["prep_1", "prep_2", "prep_3"], name="preprocessor__switch"),
    Categorical(["model_1", "model_2"], name="model__switch"),
    Categorical(["minority", "all"], name="resampler__resampler_1__sampling_strategy"),
    Integer(5, 15, name="model__model_2__max_depth")
]


### Architecture 2

In this idea, we ensemble the whole model in a single `SwitchCase` that contains several `Pipeline` object. It's more verbose (in the sense that we might have a combinatorial explosion if we want to keep all combinations of discrete options). But in simple cases it might be good enough, specially if we feel in advance that some options won't be good when combined, so we can exclude some combinations from the search by specifying the model this way. 

What is important in this approach is not to nest `Pipelines` inside each option, as this is not supported by `imblearn` `Pipeline`. 

For simplification, in the following example I've only considered three discrete global options, instead of reproducing the previous twelve options:

In [15]:
clf_2 = SwitchCase(
    cases=[
        (
            "option_1",
            ImbPipeline(
                [
                    ("resampler", SMOTE(k_neighbors=3)), 
                    ("scale", StandardScaler()), 
                    ("reduce_dims", PCA(n_components=5)), 
                    ("model", RandomForestClassifier())
                ]
            )
        ),
        (
            "option_2",
            ImbPipeline(
                [
                    ("scale", StandardScaler()), 
                    ("reduce_dims", SelectKBest(mutual_info_classif, k=5)), 
                    ("model", LogisticRegression())
                ]
            )
        ),
        (
            "option_3",
            ImbPipeline(
                [
                    ("resampler", SMOTE(k_neighbors=3)), 
                    ("scale", StandardScaler()), 
                    ("reduce_dims", SelectKBest(mutual_info_classif, k=5)), 
                    ("model", RandomForestClassifier())
                ]
            )
        ),
    ],
    switch="option_1"
)

search_space_2 = [
    Categorical(["option_1", "option_2", "option_3"], name="switch"),
    Categorical(["minority", "all"], name="option_1__resampler__sampling_strategy"),
    Integer(5, 15, name="option_1__model__max_depth"),
    Categorical(["minority", "all"], name="option_3__resampler__sampling_strategy"),
    Integer(5, 15, name="option_3__model__max_depth"),
]


## API

Let's now use the API to find the best model for the first case. 

We will calibrate the models (only the best, when refit) and we will skip some folds to increase speed. 

In [19]:
best_model, best_params, report = find_best_model(
    X=X,
    y=y,
    model=clf,
    search_space=search_space,
    verbose=False,
    k_inner=8,
    k_outer=8,
    skip_inner_folds=[2, 6],
    skip_outer_folds=[2, 6],
    n_initial_points=10,
    n_calls=10,
    calibrate="only_best",
    calibrate_params={"method": "isotonic"},
    optimizing_metric=make_scorer(roc_auc_score, average='weighted', multi_class='ovr', needs_proba=True),
    other_metrics={"acc": "accuracy"},
    skopt_func=gbrt_minimize
)

Looping over 0 outer fold
Looping over 1 outer fold
Looping over 2 outer fold
Looping over 3 outer fold
Looping over 4 outer fold
Looping over 5 outer fold
Looping over 6 outer fold
Looping over 7 outer fold


With this procedure we can obtain a lot of information. Let's start with the basics, and in other notebook we will show more advanced things. 

### Dataframe

We can create a dataframe to visualize all information of the report. 

In [20]:
report.to_dataframe()

,best,outer_kfold,model,outer_test_indexes,param__model__model_2__max_depth,param__model__switch,param__preprocessor__switch,param__resampler__resampler_1__sampling_strategy,param__resampler__switch,inner_validation_metrics__acc,inner_validation_metrics__optimizing_metric,outer_test_metrics__acc,outer_test_metrics__optimizing_metric
0,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",11,model_1,prep_1,minority,resampler_1,0.548704,0.665680,NaN,NaN
1,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10,model_1,prep_1,all,resampler_2,0.781462,0.725599,NaN,NaN
2,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",13,model_2,prep_3,minority,resampler_2,0.807226,0.803593,NaN,NaN
3,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10,model_2,prep_1,minority,resampler_1,0.737069,0.703894,NaN,NaN
4,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",8,model_2,prep_1,minority,resampler_2,0.789985,0.722436,NaN,NaN
5,True,0,CalibratedClassifierCV(base_estimator=Pipeline...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",12,model_2,prep_2,all,resampler_1,0.722815,0.810280,0.84,0.854912
6,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",9,model_1,prep_2,minority,resampler_1,0.561455,0.713804,NaN,NaN
7,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7,model_2,prep_1,all,resampler_2,0.788630,0.734129,NaN,NaN
8,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,model_2,prep_1,all,resampler_2,0.794328,0.722329,NaN,NaN
9,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",12,model_2,prep_2,all,resampler_1,0.753004,0.804796,NaN,NaN


### Params

We can find which combination of hyperparameters was the best for each outer loop. 

In [21]:
report.get_best_params()

{'resampler__switch': ['resampler_1',
  'resampler_2',
  'resampler_1',
  'resampler_2',
  'resampler_1',
  'resampler_1'],
 'preprocessor__switch': ['prep_2',
  'prep_3',
  'prep_2',
  'prep_3',
  'prep_2',
  'prep_3'],
 'model__switch': ['model_2',
  'model_2',
  'model_2',
  'model_2',
  'model_2',
  'model_2'],
 'resampler__resampler_1__sampling_strategy': ['all',
  'minority',
  'all',
  'minority',
  'all',
  'all'],
 'model__model_2__max_depth': [12, 13, 9, 9, 15, 10]}

And we can compare the previous list with the best_params of the model produced with the whole dataset. 

In [22]:
best_params

{'resampler__switch': 'resampler_2',
 'preprocessor__switch': 'prep_3',
 'model__switch': 'model_2',
 'resampler__resampler_1__sampling_strategy': 'all',
 'model__model_2__max_depth': 15}

### Outer metrics report

We can get a very basic report (mean, sd, min and max) of each metric for all outer folds. 

In [23]:
report.get_outer_metrics_report()

{'acc': {'mean': 0.8116666666666666,
  'sd': 0.021147629234082505,
  'min': 0.78,
  'max': 0.84},
 'optimizing_metric': {'mean': 0.8237939797856036,
  'sd': 0.04158393711624927,
  'min': 0.7433634638606175,
  'max': 0.8677873505643493}}